# Convolutions

Last time, we learned about how to represent images in Python with `numpy`. In this lesson, we'll learn about convolutions, specifically on image data, and how to implement convolutions. By the end of this lesson, students will be able to:  

- Define what image convolution is;
- Recognize common kernels for image convolution and their functionalities;
- Work with `numpy` arrays for representing images and kernels and applying convolutions.

In [ ]:
!pip install -q opencv-python

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def compare_two_images(im1, im2):
    """
    A helper function to display two images side by side.
    """
    _, axs = plt.subplots(1, 2, figsize=(15, 6))
    axs[0].imshow(im1, cmap="gray")
    axs[1].imshow(im2, cmap="gray")
    plt.show()

## Why Convolutions?

Not only does it have an elegant mathematical definition and important theorems associated with it (you should watch 3Blue1Brown's [video](https://www.youtube.com/watch?v=KuXjwB4LzSA) on convolutions which will be way better than what I can explain), it is also an indispensable component of a lot of *important* neural network architectures, like the U-net,

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" alt="u-net" style="width:50%"/>

which underlies the stable diffusion model for text-to-image generation.

<img src="https://ommer-lab.com/wp-content/uploads/2022/08/article-Figure3-1-1024x508.png" alt="latent-diffusion" style="width:50%"/>

## Convolution as a Mathematical Operator

Convolution defined mathematically is an operator that takes as input two functions ($f$ and $g$) and produces an third function ($f * g$) as output. Recall from last lesson that an image can be considered as a 2D function $F(x,y) = \text{pixel value at x,y coordinate}$. (Think of this $F$ as a grayscale image with one channel; for colors images with multiple channels we can work with individual channels in the same way.)

In [ ]:
dubs = cv2.imread("dubs.jpg", 0) # reads the image in as a grayscale image
plt.imshow(dubs, cmap="gray");

In [ ]:
dubs.shape, dubs.ndim, dubs.dtype

In [ ]:
# F(100, 100)
dubs[100, 100]

## Numpy Basics

Before we begin the topic, let's first learn some basic methods for creating `numpy` arrays, which might be useful when we implement convolution later in this lesson. You've seen from last lesson that you can create `numpy` arrays from lists.

In [ ]:
np.array([1, 2, 3])

In [ ]:
np.array(list(range(1, 4)))

In [ ]:
np.arange(1, 4) # Create an array with a range of elements

There are also some specialized methods for creating filled arrays.

In [ ]:
# Create an array of zeros
zeros = np.zeros((2, 3))  # 2x3 array of zeros
print(zeros)

# Create an array of ones
ones = np.ones((3, 2))  # 3x2 array of ones
print(ones)

# Create a 3D array of random numbers
random_array = np.random.rand(2, 3, 4)  # 2x3x4 array of random numbers
print(random_array)

## Image Convolutions

Let's watch 3Blue1Brown's short video on image convolution as it's worth thousands of words in terms of explaining the concept.

[![what is image convolution](https://img.youtube.com/vi/4xWpQe3G9qI/0.jpg)](https://www.youtube.com/watch?v=4xWpQe3G9qI)

In the example shown in the video we are applying a 3x3 blur filter/kernel *matrix* to the image *matrix* that is basically averaging the pixels in a 3x3 patch.
$$kernel = \begin{bmatrix}
\frac{1}{9} & \frac{1}{9} & \frac{1}{9}\\[0.3em]
\frac{1}{9} & \frac{1}{9} & \frac{1}{9}\\[0.3em]
\frac{1}{9} & \frac{1}{9} & \frac{1}{9}
\end{bmatrix}$$

Here's how we can create this kernel in Python:

In [ ]:
blur_kernel = 1 / 9 * np.ones((3, 3))
blur_kernel

Why is it averaging? Because the sum of the entries in the kernel is 1. The entries are like weights applied to a set of values.

In [ ]:
np.sum(blur_kernel), blur_kernel.sum()

The kernel needs to be flipped both left to right and upside down but in many implementations it stays unflipped and the implemented operation becomes effectively cross-correlation instead of convolution. There's usually no need to distinguish the two for image convolutions and we can just go with the option that's easier to reason about.

<img src="flip_fg.png" style="width:50%"/>

We can do this because the kernels that are used are often invariant under flipping:

In [ ]:
np.fliplr(np.flipud(blur_kernel))

In [ ]:
# not true for symmetric matrices
onetwothree = np.array([[1,2,3],[2,2,3],[3,3,3]])
print(onetwothree)
print(onetwothree.T)
onetwothree_flipped = np.fliplr(np.flipud(onetwothree))
print(onetwothree_flipped)

In [ ]:
# not to mention any matrix
random_arr = np.random.rand(3,3)
print(random_arr)
random_arr_flipped = np.fliplr(np.flipud(random_arr))
print(random_arr_flipped)

### Practice: code the convolution for images

A convolution can be thought of as a special way of looping over the pixels in an image. Instead of looping over an image one pixel at a time, a convolution loops over an image one subimage (sliced portion of the image) at a time. We call a convolution a **sliding window algorithm** because the algorithm starts at the top row, generates the first subimage for the top leftmost corner, then slides over 1 pixel to the right, and repeats the process.

The sum of the element-wise product between the subimage matrix generated at pixel $(x,y)$ and the kernel matrix becomes the new value at the convolution image's pixel $(x,y)$. It can be written as: $$(f * g) [x, y] = \sum_{i\in[n],j\in[m]} f[x-i, y-j] g[i, j]$$
where $f$ is a single-channel image and $g$ is an $n\text{ by }m$ kernel matrix.

When looping over an image, we need to consider what should happen at the boundary. There are multiple ways to handle boundary values, and below are the three modes (2D version) used in `numpy.convolve`, a [function](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html) for doing 1D convolutions.

<img src="conv_modes.png" style="width:50%"/>

And for "full" and "same", there's also subtlety in how to fill the imaginary values outside of the image boundary. Depending on your inputs and applications/goals, this could also be different. For now, we'll just consider the "valid" mode and not worry about how to handle boundary values.

Discuss with your neighbors for 30 seconds before we start: what are the sizes of the convolution results for the three modes? Write them in terms of image_width (iW), image_height (iH), kernel_width (kW), kernel_height (kH).

<details><summary>answer</summary>
<ul>
<li> full: image_width + kernel_width - 1, image_height + kernel_height - 1</li>
<li> same: image_width,                    image_height</li>
<li>valid: image_width - kernel_width + 1, image_height - kernel_height + 1</li>
</ul>
</details>

Let's now practice coding 2D convolution in Python under the "valid" mode.

**Task 1**: first, think of how to iterate over an image to obtain the subimages the same size as the kernel.  

PollEverywhere question: which of the following options do you think generate the correct subimages?

In [ ]:
def get_subimages(im, kernel):
    """
    Takes as input an image (single-channel) and a kernel/filter, both being numpy arrays.
    Iterate over im to obtain the subimages the same size as the kernel and
    return the subimages as a dictionary keyed by pixel coordinates.
    """
    assert im.ndim == 2, kernel.ndim == 2
    image_w, image_h = im.shape
    kernel_w, kernel_h = kernel.shape
    conv_im_w = image_w - kernel_w + 1
    conv_im_h = image_h - kernel_h + 1

    subimages = {}
    for x in range(conv_im_w):
        for y in range(conv_im_h):
            ## option 1
            subimage = im[x - 1:x + kernel_w - 1, y - 1:y + kernel_h - 1]
            ## option 2
            # subimage = im[x    :x + kernel_w,     y:  y + kernel_h]
            ## option 3
            # subimage = im[x    :x + kernel_w + 1, y:  y + kernel_h + 1]
            ## option == 4:
            # subimage = im[x + 1:x + kernel_w + 1, y + 1:y + kernel_h + 1]
            ## option == 5:
            # subimage = im[y    :y + kernel_h,     x:  x + kernel_w]
            subimages[x, y] = subimage
    return subimages

In [ ]:
# setting up our test case
small_dubs = np.copy(dubs[50:100, 300:350])
compare_two_images(small_dubs, small_dubs[30:33,20:23])

In [ ]:
# test out the method
small_dubs_subimages = get_subimages(small_dubs, blur_kernel)
compare_two_images(small_dubs_subimages[(30, 20)], small_dubs[30:33,20:23]) # visual comparison is usually the most helpful

assert small_dubs_subimages[(30, 20)].shape == blur_kernel.shape, "subimage does not have same size as the kernel"
assert np.allclose(small_dubs_subimages[(30, 20)], small_dubs[30:33,20:23]), "subimage does not match the expected subimage"

**Task 2**: compute the value at $x,y$ and assign to the array representing the convolution result.

PollEverywhere question: this value is a sum of an elementwise product; fill in the code for the product.

In [ ]:
def im_conv(im, kernel):
    """
    Takes as input an image (single-channel) and a kernel/filter, both being numpy arrays.
    Returns the convolution of the kernel over the image under the "valid" mode
    where the image size shrinks by the size of the kernel.
    """
    assert im.ndim == 2, kernel.ndim == 2
    image_w, image_h = im.shape
    kernel_w, kernel_h = kernel.shape
    conv_im_w = image_w - kernel_w + 1
    conv_im_h = image_h - kernel_h + 1
    conv_image = np.zeros((conv_im_w, conv_im_h))

    for x in range(conv_im_w):
        for y in range(conv_im_h):
            ...

    return conv_image

Let's see what the blur kernel does to the dubs image.

In [ ]:
dubs_blurred = im_conv(dubs, blur_kernel)
compare_two_images(dubs, dubs_blurred)

What if we make the kernel size larger? Discuss with your neighbors what might be the expected result.

In [ ]:
blur_kernel_5x5 = np.ones((5,5))
blur_kernel_5x5 /= blur_kernel_5x5.size
blur_kernel_5x5

In [ ]:
dubs_blurred_5x5 = im_conv(dubs, blur_kernel_5x5)
compare_two_images(dubs, dubs_blurred_5x5)
compare_two_images(dubs_blurred, dubs_blurred_5x5)

### More filters/kernels

So far we've seen averaging filters/box blur filters. Next let's try to deduce what a kernel might do to an image.

#### Kernel 1

In [ ]:
kernel = np.zeros((3,3))
kernel[1,1] = 1
kernel

In [ ]:
conv_im = im_conv(dubs, kernel)
compare_two_images(dubs, conv_im)

In [ ]:
dubs.shape, conv_im.shape

In [ ]:
assert np.allclose(dubs[1:-1, 1:-1], conv_im)

In [ ]:
identity_kernel = np.zeros((3,3))
identity_kernel[1,1] = 1
identity_kernel

#### Kernel 2

In [ ]:
kernel = 2 * identity_kernel - blur_kernel
kernel

In [ ]:
conv_im = im_conv(dubs, kernel)
compare_two_images(dubs, conv_im)

You might think that it looks darker, but looking closer it also seems that the hair strands stand out more. Let's take a look at the result of applying convolution to the identity_kernel with the blur_kernel subtracted from it.

In [ ]:
conv_image_id_sub_blur = im_conv(dubs, identity_kernel - blur_kernel)
plt.imshow(conv_image_id_sub_blur, cmap="gray");

It's kind of hard to see so let's do some thresholding to turn all the gray pixels black and the others white to make the pattern easier to see.

In [ ]:
plt.imshow(cv2.threshold(conv_image_id_sub_blur, 15, 200, cv2.THRESH_BINARY)[1], cmap="gray");

Intuitively, this is subtracting the blurred image from the original image, leaving with us the places where the pixels change colors. You could say that these regions are the important silhouettes that let us recognize the pattern in the image. Now if we apply the sum of the identity_kernel and this difference (identity_kernel - blur_kernel) as a kernel to the image, it's like adding these regions back to the original image, which further stresses them.

In [ ]:
sharpening_kernel = 2 * identity_kernel - blur_kernel
sharpening_kernel

### Kernel 3

In [ ]:
gaussian_blur_kernel = np.array([[1,4,6,4,1]]) * np.array([[1,4,6,4,1]]).T
print(gaussian_blur_kernel)
gaussian_blur_kernel = gaussian_blur_kernel / 256
gaussian_blur_kernel

In [ ]:
conv_im = im_conv(dubs, gaussian_blur_kernel)
compare_two_images(dubs, conv_im)

### OpenCV and some examples

OpenCV is a python library that provides many useful image processing methods, including convolution filters. Note that there's some difference between the implementations that opencv uses and we have introduced in class but the overall result should be qualitatively similar.

In [ ]:
# Use the sharpening kernel
sharpened_image = cv2.filter2D(dubs, -1, sharpening_kernel)
plt.imshow(sharpened_image, cmap="gray");

In [ ]:
# Use the (approximate) Gaussian blur
blurred_filtered_image = cv2.filter2D(dubs, -1, gaussian_blur_kernel)
# alternatively, use opencv's Gaussian blur
# blurred_filtered_image = cv2.GaussianBlur(dubs, (5, 5), 0)
plt.imshow(blurred_filtered_image, cmap="gray");

In [ ]:
# Define an edge detection filter kernel
edge_kernel = np.array([[-1, -1, -1],
                        [-1, 8, -1],
                        [-1, -1, -1]])
edge_filtered_image = cv2.filter2D(dubs, -1, edge_kernel)
plt.imshow(edge_filtered_image, cmap="gray");

Here's an example of how to do edge detection of an image in opencv. You can read the [documentation](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga2a671611e104c093843d7b7fc46d24af)/[tutorial](https://docs.opencv.org/3.4/da/d22/tutorial_py_canny.html) for what the parameters actually do.

In [ ]:
# Canny Edge Detection
edges_orig = cv2.Canny(dubs, 100, 200)
edges_blurred = cv2.Canny(blurred_filtered_image, 100, 200)
compare_two_images(edges_orig, edges_blurred)

There are many more useful methods for doing image processing and computer vision tasks, not just convolutions, in opencv. For example, when preparing image data for ML model training, it's often necessary to transform the images to be a certain dimension, split off the channels, or compute simple features off of the images and put them into a different vector. For anyone working with image data for their projects, it is a great tool.